In [202]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
dups = pd.read_csv('103020_duplicates.csv')
no_dups = pd.read_csv('103020_no_duplicates.csv')
pd.set_option('display.max_rows', None)


# dropping extra columns that pull from the google sheets download of '103020_no_duplicates.csv'
no_dups.drop(no_dups.iloc[:, 16:], inplace = True, axis = 1)

# pulling a csv from google sheets or excel causes whitespace
# in certain column headers (no idea why); need to strip whitespace
# from all column names before calling any column name
no_dups = no_dups.rename(columns=lambda x: x.strip())

This section of code creates the duplicate-based DataFrame containing COUNTS

In [203]:
# # Creating DataFrame "count" from the dups (duplicates included)
# Sets Form Date to datetime format; "Can only use .dt accessor
# with datetimelike values" (that's why form_date needs to be converted
# to datetime first, even though it looks redundant)
dups['form_date']= pd.to_datetime(dups['form_date'])
# convert to dt
dups['form_date']= dups['form_date'].dt.date
# Creating "count" variable, count of projects by day
count = dups.groupby(['form_date']).size().sort_values(ascending=False)

In [204]:
# Casts count to a dataframe
count = pd.DataFrame(data = count)
# Resets index
count.reset_index(inplace=True)
# Renames "0" column
count = count.rename(columns = {0: "count"})

count['count'] = count['count'].fillna(0)


count.head()

,form_date,count
0,2020-06-23,127
1,2020-05-20,119
2,2020-10-22,115
3,2020-10-27,113
4,2020-06-10,112


This section of code creates the non-duplicate dataframe containing REVENUE

In [205]:
# # Creating DataFrame "rev" from no_dups (duplicates NOT included)
# Sets form_date to datetime format; "Can only use .dt accessor
# with datetimelike values" (that's why form_date needs to be converted
# to datetime first, even though it looks redundant)
no_dups['form_date']= pd.to_datetime(no_dups['form_date'])
# convert to dt
no_dups['form_date']= no_dups['form_date'].dt.date
# creates variable "rev"; sum of gross amount by form date and filtered by "stage" == "Closed - Won"
rev = no_dups[no_dups['stage'] == 'Closed - Won'].groupby('form_date')['gross_amount'].sum()

In [206]:
# creates variable "rev"; sum of gross amount by form date and filtered by "stage" == "Closed - Won"
# rev = no_dups[no_dups['stage'] == 'Closed - Won'].groupby('form_date')['gross_amount'].sum()
# casts rev to a dataframe
rev = pd.DataFrame(data = rev)
# resets index
rev.reset_index(inplace = True)
# casts gross_amount to int64
rev['gross_amount'] = rev['gross_amount'].astype(np.int64)
# fills nans
rev['gross_amount'] = rev['gross_amount'].fillna(0)

rev.head()

,form_date,gross_amount
0,2020-02-18,49500
1,2020-02-19,37746
2,2020-02-20,115605
3,2020-02-21,12340
4,2020-02-24,23220


In [207]:
# Redefine count and rev for clarity
left = count
right = rev
# Joining count and rev on form_date
df = left.merge(right, on='form_date', how='left')

df['gross_amount'] = df['gross_amount'].fillna(0)
df['gross_amount'] = df['gross_amount'].astype(np.int64)
df

,form_date,count,gross_amount
0,2020-06-23,127,297798
1,2020-05-20,119,408783
2,2020-10-22,115,305135
3,2020-10-27,113,418435
4,2020-06-10,112,415881
5,2020-06-29,109,611287
6,2020-06-30,107,313380
7,2020-06-25,107,686795
8,2020-06-24,105,271440
9,2020-05-28,102,504403


In [208]:
# Defining x and y
x = df['count'].to_numpy().reshape((-1, 1))
y = df['gross_amount'].to_numpy()

In [209]:
# Assigning model type to "model"
model = LinearRegression()

In [210]:
# Fitting model to x and y
model.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [211]:
model = LinearRegression().fit(x, y)


In [212]:
print('intercept:', model.intercept_,
     'slope:', model.coef_,
     'coefficient of determination:', r_sq, sep='\n')

intercept:
8505.864893028658
slope:
[4602.54934958]
coefficient of determination:
0.3728110517027784


In [213]:
obj = np.array([46, 120]).reshape((-1, 1))

In [214]:
y_pred = model.predict(obj)

In [215]:
print('predicted response:', y_pred, sep='\n')

predicted response:
[220223.13497388 560811.78684308]


In [89]:
z = y_pred.astype(int)
z

array([316864, 835532])